In [1]:
##AWS 람다 테스트용 코드


import requests
url = "https://xxanatchmvujwbl7cz2nguy3440hgaoq.lambda-url.ap-northeast-2.on.aws/"
r = requests.post(url, json={"issue_key": "P2-70735", "update": True})
print(r.status_code, r.text)

200 {"issue_key":"P2-70735","update_payload":{"summary":"[Test] [System Menu] 에디터 상에서 언어 변경 콤보 박스 관련 데이터가 누락되어 있는 현상 \/ Language change combo box related data missing in the editor","description":"Observed\/관찰 결과\n에디터 상에서 언어 변경 콤보 박스 관련 데이터가 누락되어 있는 현상이 발생합니다.\n{color:#4c9aff}An issue occurs in which language change combo box related data is missing in the editor.{color}\n\n* game_dev 클라이언트 상에선 언어 콤보 박스가 정상적으로 출력되지만, 에디터 상에선 언어 콤보 박스 관련 데이터가 누락되어 있는 상황입니다.\n{color:#4c9aff}On the game_dev client the language combo box is displayed normally, but in the editor the language combo box related data is missing.{color}\n\n!image-20251118-030747.png|width=50%,alt=\"image-20251118-030747.png\"!\n\nExpected\/기대 결과\n* 에디터 상에서 언어 변경 콤보 박스 관련 데이터가 누락되지 않아야 합니다.\n{color:#4c9aff}The language change combo box related data should not be missing in the editor.{color}\n\nNote\/참고\n* 본 이슈는 [~accountid:712020:e60c6ea3-056a-4472-945b-102aa327a99e] 님으로부터 제보 받은 이슈로 빌드 안전을 위해 수정 작업을 위한 버그 티켓인 점 참고 부탁드립니다.\n{col